In [1]:
import pandas as pd

dfAuthors = pd.read_csv('DBPediaAuthorsReducedSample.csv', sep=',', low_memory=False)

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

#dfAuthorsViaf = pd.DataFrame()
dfAuthorsViaf = pd.read_csv('VIAFDataAuthors.csv', sep=',', low_memory=False)
display(dfAuthors.head())

,URI,birthDate,deathDate,gender,label,links,work
0,URI,http://dbpedia.org/ontology/birthDate,http://dbpedia.org/ontology/work,http://xmlns.com/foaf/0.1/gender,http://www.w3.org/2000/01/rdf-schema#label,www.w3.org/2002/07/owl#sameAs,http://dbpedia.org/ontology/work
1,URI,XMLSchema#date,XMLSchema#list,XMLSchema#string,XMLSchema#string,XMLSchema#list,XMLSchema#list
2,http://www.w3.org/2002/07/owl#Thing,http://www.w3.org/2001/XMLSchema#date,https://www.w3.org/2001/XMLSchema#list,http://www.w3.org/2001/XMLSchema#string,http://www.w3.org/2001/XMLSchema#string,https://www.w3.org/2001/XMLSchema#list,https://www.w3.org/2001/XMLSchema#list
3,http://dbpedia.org/resource/Abbie_Hoffman,1936-11-30,1989-04-12,male,Abbie Hoffman,\thttp://viaf.org/viaf/79408884\thttp://www.wi...,\tSteal_This_Book\tWoodstock_Nation_(book)
4,http://dbpedia.org/resource/Adam_Oehlenschläger,1779-11-14,1850-01-20,male,Adam Oehlenschläger,\thttp://dbpedia.org/resource/Adam_Oehlenschlä...,NaN


In [27]:
import rdflib
from rdflib import URIRef, Literal
import pandas as pd

import re

def getWorkTitle(identifier):
    g = rdflib.Graph()
    g.load(''.join([identifier, '/rdf.xml']))
    
    for s,p,o in g:
        if(str(p) == 'http://schema.org/name'):
            return str(o)
        
    

def convertResultToDataFrame( identifier ):
  #  try:                         
        g = rdflib.Graph()
        g.load(''.join([identifier, '/rdf.xml']))
                             
        author = {}
        author['uri'] = identifier
        search_attributes = ['birthDate', 'deathDate', 'name', 'gender', 'notableWork']
        gender = {}
        gender['http://www.wikidata.org/entity/Q6581097'] = 'male'
        gender['http://www.wikidata.org/entity/Q6581072'] = 'female'
        gender['http://www.wikidata.org/entity/Q1097630'] = 'intersex'
        
        
        for s,p,o in g:
            predicate = p.split('/')[-1].strip()
            if predicate in search_attributes:
                add = False
            
                if type(o) is rdflib.term.Literal and not type(o.datatype) is rdflib.term.URIRef:
                    tmp_value = re.sub('\W', '', o.value.strip())
                    if  len(re.findall('[\0-\200]', tmp_value)) != 0 :
                        value = o.value
                        add = True
                else:
                    value = o
                    add = True
            
                if add:
                    
                    if predicate == 'notableWork' :
                        work = getWorkTitle(value)
                        if predicate not in author:
                            author[predicate] = work
                        elif(not value in author[predicate].split('\t ')):
                            author[predicate] = '\t '.join([author[predicate], work])
                            
                    elif predicate not in author:
                        if predicate == 'gender':
                            author[predicate] = gender[str(value)]
                        else:
                            author[predicate] = value

#    except:
 #       display(identifier)
  #      return pd.DataFrame()
        return pd.DataFrame(author, index=[0])

dfAuthorsViaf = pd.DataFrame()
for row in dfAuthors['links']:
    for link in row.split('\t'):
        if 'http://viaf.org/' in link:
            if link not in dfAuthorsViaf['identifier'].values:
                dfAuthorsViaf = dfAuthorsViaf.append(convertResultToDataFrame(link), ignore_index=True, sort=True)
display(dfAuthorsViaf)

,birthDate,deathDate,gender,name,notableWork,uri
0,1936-11-30,1989-04-12,male,Abbie Hoffman,Soon to be a major motion picture\t Revolution...,http://viaf.org/viaf/79408884
1,1779-11-14,1850-01-20,male,Adam Oehlenschläger,Correggio\t Vaulundurs saga\t Nordens guder\t ...,http://viaf.org/viaf/27105324
2,1937-01-01,1995-08-30,male,Adam Wiśniewski-Snerg,Robot\t Według łotra\t Nagi cel,http://viaf.org/viaf/85523915
3,1929-05-16,2012-03-27,female,Adrienne Rich,"Rich, Adrienne,\t Rich, Adrienne,\t Rich, Adri...",http://viaf.org/viaf/16565
4,1785-03-07,1873-05-22,male,Alessandro Manzoni,"Manzoni, Alessandro (1785-1873).\t Manzoni\t M...",http://viaf.org/viaf/14356
5,1913-12-18,1987-09-30,male,Alfred Bester,Deceivers\t Dark side of the earth\t Psycho sh...,http://viaf.org/viaf/56605956
6,NaN,NaN,NaN,NaN,NaN,http://viaf.org/viaf/218763367
7,1621-03-31,1678-08-16,male,Andrew Marvell,"Garden\t Poems\t Marvell, Andrew,\t Upon Apple...",http://viaf.org/viaf/39381003
8,1928-10-31,2012-06-20,male,Andrew Sarris,NaN,http://viaf.org/viaf/79132061
9,1896-02-18,1966-09-28,male,André Breton,"Les vases communicants\t Breton, Andre,\t Pa...",http://viaf.org/viaf/49222341
